## Periods in Rubin compared to ZTF

In this notebook we will compare periods for objects in Rubin with those we get from ZTF.

In [ ]:
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tempfile

from astropy.timeseries import LombScargle
from dask.distributed import Client
from nested_pandas import NestedDtype
from pathlib import Path

In [ ]:
def cast_nested(df, columns):
    return df.assign(
        **{
            col: df[col].astype(NestedDtype.from_pandas_arrow_dtype(df.dtypes[col]))
            for col in columns
        },
    )

In [ ]:
tmp_path = tempfile.TemporaryDirectory()
client = Client(n_workers=4, threads_per_worker=1, local_directory=tmp_path.name)

### Load Rubin and ZTF

In [ ]:
drp_release = "w_2025_10"
base_dir = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats")
hats_dir = base_dir / drp_release

In [ ]:
rubin_variables = lsdb.read_hats("rubin_variables")
# We use the `cast_nested` utility method to cast columns into the NestedFrame type
rubin_variables = rubin_variables.map_partitions(cast_nested, columns=["forcedSource"])

We will read `object_lc_x_ps1` which we had already computed for the result of the crossmatch of Rubin with PS1.

In [ ]:
object_lc_x_ztf = lsdb.read_hats(
    hats_dir / "object_lc_x_ztf_dr22", columns=["objectId", "ztf_lc"]
)
# We use the `cast_nested` utility method to cast columns into the NestedFrame type
object_lc_x_ztf = object_lc_x_ztf.map_partitions(cast_nested, columns=["ztf_lc"])

We can now crossmatch our catalog of Rubin periods with `object_lc_x_ztf` to get the aggregated data that we need:

In [ ]:
variables_catalog = rubin_variables.join(
    object_lc_x_ztf, left_on="objectId", right_on="objectId", suffixes=("_rubin", "")
)

In [ ]:
# Compute the result for the 10 variable objects
variables = variables_catalog.compute()

### Calculate periods for PS1 and compare with Rubin's

In [ ]:
def calculate_ztf_periods(true_period, ztf_time, ztf_flux):
    # define a freq grid to search around the true period/frequency
    true_freq = 1 / true_period
    frequency = np.linspace(0.9 * true_freq, 1.1 * true_freq, 100)
    ztf_power = LombScargle(ztf_time, ztf_flux).power(frequency)
    ztf_maxpower = np.max(ztf_power)
    ztf_period = 1 / frequency[np.argmax(ztf_power)]
    return {"maxpower_ztf": ztf_maxpower, "period_ztf": ztf_period}

In [ ]:
# Each object has a single-band light curve
[np.unique(lc["filterid_ztf"]) for lc in variables["ztf_lc"]]

In [ ]:
periods = variables.reduce(
    calculate_ztf_periods, "true_period_rubin", "ztf_lc.hmjd_ztf", "ztf_lc.mag_ztf"
)
variables = pd.concat([variables, periods], axis=1)

### Plot original and phased light curves

Now let's plot light curves for each object on both surveys:

In [ ]:
# Rubin has "ugrizy" filters
COLORS = {
    "u": "#56b4e9",
    "g": "#009e73",
    "r": "#f0e442",
    "i": "#cc79a7",
    "z": "#d55e00",
    "y": "#0072b2",
}

# PanSTARRS has "gri" filters [0..2]
ZTF_COLORS = {i: color for i, color in enumerate("gri")}

In [ ]:
def plot_ztf(ax, lc, period):
    mjd_col = "hmjd_ztf"
    mag_col = "mag_ztf"
    magerr_col = "magerr_ztf"
    # Plot original light curve
    ax[0].set_title("ZTF - Original")
    mags = _plot_ztf_lc(ax[0], lc, mag_col, magerr_col, x_name=mjd_col, x_label="MJD")
    # Plot phased light curve
    ax[1].set_title(f"ZTF - Phase folded ({period})")
    lc = lc.assign(
        phase=(lc[mjd_col] - lc[mjd_col].loc[lc[mag_col].idxmax()]) % period / period
    )
    _plot_ztf_lc(ax[1], lc, mag_col, magerr_col, x_name="phase", x_label="Phase")
    scale_y_axis(ax, mags)
    scale_x_axis(ax, lc[mjd_col], lc["phase"], delta=200)


def plot_rubin(ax, lc, period):
    mjd_col = "midpointMjdTai"
    mag_col = "psfMag"
    magerr_col = "psfMagErr"
    # Plot original light curve
    ax[0].set_title("Rubin - Original")
    mags = _plot_rubin_lc(ax[0], lc, mag_col, magerr_col, x_name=mjd_col, x_label="MJD")
    # Plot phased light curve
    ax[1].set_title(f"Rubin - Phase folded ({period})")
    lc = lc.assign(
        phase=(lc[mjd_col] - lc[mjd_col].loc[lc[mag_col].idxmax()]) % period / period
    )
    _plot_rubin_lc(ax[1], lc, mag_col, magerr_col, x_name="phase", x_label="Phase")
    scale_y_axis(ax, mags)
    scale_x_axis(ax, lc[mjd_col], lc["phase"], delta=5)


def _plot_ztf_lc(ax, lc, mag_col, magerr_col, x_name, x_label):
    mag_values = []
    for filterid, band in ZTF_COLORS.items():
        band_lc = lc.query(f"filterid_ztf == {filterid}")
        mag, magerr = band_lc[mag_col], band_lc[magerr_col]
        ax.errorbar(
            band_lc[x_name],
            mag,
            magerr,
            fmt="o",
            label=band,
            color=COLORS[band],
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        mag_values.extend(mag.dropna().values)
    ax.set_xlabel(x_label)
    ax.set_ylabel("Magnitude (AB)")
    ax.invert_yaxis()
    ax.legend(loc="lower right", fontsize=12)
    return mag_values


def _plot_rubin_lc(ax, lc, mag_col, magerr_col, x_name, x_label):
    mag_values = []
    for band, color in COLORS.items():
        band_lc = lc.query(f"band == '{band}'")
        mag, magerr = band_lc[mag_col], band_lc[magerr_col]
        ax.errorbar(
            band_lc[x_name],
            mag,
            magerr,
            fmt="o",
            label=band,
            color=color,
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        mag_values.extend(mag.dropna().values)
    ax.set_xlabel(x_label)
    ax.set_ylabel("Magnitude (AB)")
    ax.invert_yaxis()
    ax.legend(loc="lower right", fontsize=12)
    return mag_values


def scale_x_axis(ax, mjd_values, phase_values, delta):
    # Apply limits to the mjd axis
    xmin, xmax = np.nanmin(mjd_values), np.nanmax(mjd_values)
    ax[0].set_xlim(xmin - delta, xmax + delta)
    # Apply limits to the phase axis
    xmin, xmax = np.nanmin(phase_values), np.nanmax(phase_values)
    ax[1].set_xlim(xmin, xmax)


def scale_y_axis(ax, all_mags):
    # Apply limits to all columns in the row
    ymin, ymax = np.nanmin(all_mags), np.nanmax(all_mags)
    lower_bound = np.quantile(all_mags, 0.1)
    upper_bound = np.quantile(all_mags, 0.9)
    for i in range(2):
        # Keep magnitude inverted
        ax[i].set_ylim(lower_bound - 0.2, upper_bound + 0.2)
    # print("Min | Max:", ymin, ymax)
    # print("10% | 90%:", lower_bound, upper_bound)

In [ ]:
for index_rubin, row in variables.iterrows():
    fig, ax = plt.subplots(2, 2, figsize=(12, 8), sharey="col")

    ra, dec = row.ra_rubin, row.dec_rubin
    true_period = row["true_period_rubin"]
    ztf_period = round(row["period_ztf"], 5)
    rubin_period = round(row["period_rubin"], 5)

    title = f"{drp_release} | RA={ra:.5f}, Dec={dec:.5f} (T={true_period})"
    fig.suptitle(title, fontsize=16)

    plot_ztf(ax[0], row["ztf_lc"], ztf_period)
    plot_rubin(ax[1], row["forcedSource_rubin"], rubin_period)

    plt.tight_layout()
    plt.show()

In [ ]:
client.close()
tmp_path.cleanup()